# LangChain RAG App 🦜

Based on our problem statement, we need to build a customer service chatbot that can answer questions based on our product documentation.This is the first itteration of our app and will be itterated on throughout this course.

## Load Libraries 🧑‍💻

- AzureChatOpenAI: Allows us to authenticate and interact with our GPT4o model

- AzureAISearchRetriever: An interface that returns documents given an unstructured query using Azure Search

- RunnablePassthrough: Allows you to pass inputs unchanged 

- ChatPromptTemplate: Creates a prompt template to help translate user input and parameters into instructions for a language model

- StrOutputParser: Ensures our LLM output is in the form of a string, making it more deterministic 

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_openai import AzureChatOpenAI
from langchain_community.retrievers import AzureAISearchRetriever
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

## Initialize GPT4o 🤖

Let's connect in to our existing Azure OpenAI instance and more specifically, our GPT4o model.

In [ ]:
model = AzureChatOpenAI(
    azure_deployment="gpt4o",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-01"
)

## Initialize Azure Search Retriever 🐕

A retriever is an interface that returns documents given an unstructured query. In our case, we want to retrieve product information from Azure Search.

This is going to help with the 'R' part in RAG or the retrieval part.

In [ ]:
retriever = AzureAISearchRetriever(
    content_key="content", top_k=5, index_name="products", api_key=os.getenv("AZURE_SEARCH_KEY"), service_name="genai-on-azure-search"
)

## Create our RAG Prompt Template 🗣️

We give our bot a persona as a 'Q&A bot' and instruct it to 'answer questions on the products we sell'.

The prompt template takes into two parameters:

1. context: the documents returned from the Azure Search retriever

2. question: the users original question

This is going to help with the 'A' part of RAG or the augmentation step.

In [ ]:
customer_bot_prompt = """You are a Q&A bot and your job is to answer questions on the products we sell
Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(customer_bot_prompt)

## Create LCEL Chain ⛓️

Below we leverage the LangChain expression language to 'chain' together our retriever, prompt, LLM (ie: gpt4o), and the StrOutputParser

In [ ]:
retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

## Question #1 ❓

In [ ]:
query = "how much is the home theater system?"
print(retrieval_chain.invoke(query))

## Question #1 Follow-up ❓

In [ ]:
query = "are you sure? can you check again?"
print(retrieval_chain.invoke(query))

## Question #2 ❓

In [ ]:
query = "what is the cheapest pair of headphones?"
print(retrieval_chain.invoke(query))